In [116]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LinearRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from nltk.corpus import wordnet
from datetime import datetime, date
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
%matplotlib inline

[nltk_data] Downloading package wordnet to /home/ilja/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [81]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.shape

(51815, 43)

In [40]:
train_X, valid_X = train_test_split(train, test_size=0.3, random_state=42)
#train_X = train

In [41]:
train_X.iloc[:5, :10]

,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access
17802,20145777,Double in a friendly house,A double bedroom in a cozy and modern apartm...,NaN,A double bedroom in a cozy and modern apartm...,none,NaN,NaN,NaN,NaN
26668,22630537,London flat with great transport links,"Light, airy and modern one bedroom flat 2 mins...",NaN,"Light, airy and modern one bedroom flat 2 mins...",none,NaN,NaN,NaN,NaN
41498,27355619,"Studio Apartments, 5 mins to Kings Cross Station!","Based in high quality student accommodation, o...",What To Expect: * 16/17m2 Studio with modern f...,"Based in high quality student accommodation, o...",none,As one of Central London’s most vibrant and ne...,• Do you have free WiFi? Yes we have super fas...,You will be in an incredible position for gett...,"You will enjoy your own studio, with a private..."
20649,21011236,"Big, Beautiful, Sunny, West Kensington TWIN Room","Big Beautiful, airy, West Kensington TWIN room...",This is a pretty terraced house in a great are...,"Big Beautiful, airy, West Kensington TWIN room...",none,We’re in a fantastic location - close to many ...,We lay on a good breakfast that guests serve t...,NaN,The room is exclusively yours while you're her...
33454,24754494,Dashing 1BR in Bayswater by Sonder,"At this Sonder, you'll love the chic decor, st...",Every booking is instantly confirmed. Every ca...,"At this Sonder, you'll love the chic decor, st...",none,"Your Sonder is in a beautiful dwelling, conver...","This Sonder does not have air conditioning, bu...",Your Sonder is located at the end of a cul-de-...,NaN


In [42]:
train_X.iloc[:5, 10:19]

,interaction,house_rules,host_id,host_since,host_about,host_response_time,host_response_rate,host_is_superhost,host_has_profile_pic
17802,NaN,"I have a ""No shoe"" policy in the house. When m...",141917708,2017-07-22,NaN,NaN,NaN,f,t
26668,NaN,NaN,76064729,2016-06-06,NaN,a few days or more,40%,f,t
41498,Our reception can help you out with anything y...,NaN,205450683,2018-07-27,NaN,within an hour,100%,t,t
20649,I'm Matthew and I live here with my son Alex. ...,The room is for a maximum of two people and on...,144406049,2017-08-05,My name's Matt and I live in the house with my...,NaN,NaN,f,t
33454,"Our concierge is available by phone, email, or...",House Rules 1. All bookings require a security...,12243051,2014-02-14,Sonder provides everything you need for an exc...,within an hour,98%,f,t


In [43]:
train_X.iloc[:5, 19:29]

,host_identity_verified,neighbourhood_cleansed,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms
17802,f,Hackney,E8 2NN,51.550560,-0.073584,f,Apartment,Private room,2,1.0
26668,f,Wandsworth,SW11 2HQ,51.463721,-0.174834,f,Apartment,Entire home/apt,4,1.0
41498,f,Islington,WC1X 9AX,51.529753,-0.115655,t,Serviced apartment,Entire home/apt,2,1.0
20649,f,Hammersmith and Fulham,W14 9PS,51.485577,-0.206449,t,Townhouse,Private room,2,1.0
33454,f,Westminster,W23JQ,51.512930,-0.186348,t,Apartment,Entire home/apt,2,1.0


In [44]:
train_X.iloc[:5, 29:38]

,bedrooms,beds,bed_type,amenities,square_feet,security_deposit,cleaning_fee,guests_included,extra_people
17802,1.0,1.0,Real Bed,"{TV,Wifi,Kitchen,""Buzzer/wireless intercom"",He...",NaN,NaN,NaN,1,0.0
26668,1.0,1.0,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,""Pets allowed"",Bre...",NaN,NaN,25.0,2,20.0
41498,0.0,1.0,Real Bed,"{TV,Wifi,Kitchen,Elevator,Heating,Washer,Dryer...",NaN,80.0,45.0,1,0.0
20649,1.0,2.0,Real Bed,"{TV,Wifi,Kitchen,""Paid parking off premises"",B...",NaN,NaN,12.0,1,11.0
33454,1.0,1.0,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,Heating,Washer,Dry...",NaN,300.0,63.0,2,5.0


In [45]:
train_X.iloc[:5, 38:]

,minimum_nights,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,price
17802,1,flexible,f,f,90.0
26668,1,moderate,f,f,75.0
41498,3,flexible,f,f,89.0
20649,1,moderate,f,f,60.0
33454,2,strict_14_with_grace_period,f,f,169.0


In [46]:
train_X.isna().sum()

id                                      0
name                                    4
summary                              1264
space                               12918
description                           721
experiences_offered                     0
neighborhood_overview               15101
notes                               22652
transit                             15413
access                              16399
interaction                         17277
house_rules                         16585
host_id                                 0
host_since                              6
host_about                          16537
host_response_time                  14507
host_response_rate                  14508
host_is_superhost                       6
host_has_profile_pic                    6
host_identity_verified                  6
neighbourhood_cleansed                  0
zipcode                              1020
latitude                                0
longitude                         

In [47]:
train_X.room_type.value_counts(dropna=False)

Entire home/apt    20776
Private room       15129
Shared room          365
Name: room_type, dtype: int64

In [48]:
long_text_cols = ['name', 'summary', 'space', 'description', 'neighborhood_overview', 'notes', 'transit',
                 'access', 'interaction', 'house_rules', 'host_about', 'amenities']
categoric_cols = ['host_response_time', 'property_type', 'bed_type', 'cancellation_policy',
                  'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified',
              'is_location_exact','require_guest_profile_picture', 'require_guest_phone_verification', 'room_type']
number_cols = ['latitude', 'longitude', 'accommodates', 'bathrooms',
              'bedrooms', 'beds', 'guests_included', 'extra_people',
              'minimum_nights']
special_cols = ['host_since', 'place_price', 'security_deposit', 'cleaning_fee']

In [49]:
agg_place = train_X.groupby('neighbourhood_cleansed')['price'].mean().reset_index()
agg_place.rename({'price': 'place_price'}, axis=1, inplace=True)

In [50]:
num_col_medians = dict()
for col in number_cols:
    num_col_medians[col] = train[col].median()

In [91]:
encoder = OneHotEncoder(handle_unknown='ignore')
for col in categoric_cols:
    train[col].fillna('None', inplace=True)
encoder.fit(train[categoric_cols]);

In [92]:
def features_transform(df):
    new_df = df.copy()
    new_df.host_since.fillna('2018-01-01', inplace=True)
    new_df['host_since'] = new_df.host_since.apply(lambda s: (date.today() - datetime.strptime(s, '%Y-%m-%d').date()).days // 30)
    new_df = new_df.merge(agg_place, how='left', on='neighbourhood_cleansed')
    new_df.place_price.fillna(115, inplace=True)
    new_df.security_deposit.fillna(0, inplace=True)
    new_df.cleaning_fee.fillna(0, inplace=True)
    new_df.drop(['experiences_offered', 'host_id', 'host_response_rate', 'neighbourhood_cleansed',
               'zipcode', 'square_feet'], axis=1, inplace=True)
    for col in number_cols:
        new_df[col].fillna(num_col_medians[col], inplace=True)
    for col in categoric_cols:
        new_df[col].fillna('None', inplace=True)
    df_dumm = pd.DataFrame(encoder.transform(new_df[categoric_cols]).toarray(), columns=encoder.get_feature_names())
    new_df = pd.concat([new_df, df_dumm], axis=1)
    return new_df

In [93]:
X_train_num = features_transform(train_X).drop(long_text_cols + categoric_cols + ['price'], axis=1)
X_valid_num = features_transform(train_X).drop(long_text_cols + categoric_cols + ['price'], axis=1)

In [94]:
X_test_num = features_transform(test).drop(long_text_cols + categoric_cols, axis=1)

In [72]:
pipeline = Pipeline([('liner', StandardScaler()),
                     ('clf', LinearRegression())])
forest = RandomForestRegressor(n_estimators=50)

In [73]:
cvs = cross_val_score(forest, X_train_num, train_X.price, cv=3, scoring='neg_mean_squared_error')

In [74]:
cvs

array([-26854.63350263, -30801.29442031, -25779.33867811])

In [77]:
forest.fit(X_train_num, train_X.price)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
forest.feature_importances_ture_importances_ture_importances_

In [95]:
X_test_num['price'] = forest.predict(X_test_num.drop('id', axis=1))

In [98]:
for col in long_text_cols:
    train_X[col].fillna('', inplace=True)
    valid_X[col].fillna('', inplace=True)
    test[col].fillna('', inplace=True)

In [99]:
train_X['all_text'] = train_X[long_text_cols].apply(lambda s: ' '.join(s), axis=1)
valid_X['all_text'] = valid_X[long_text_cols].apply(lambda s: ' '.join(s), axis=1)
test['all_text'] = test[long_text_cols].apply(lambda s: ' '.join(s), axis=1)

In [100]:
stemmer = PorterStemmer()
train_X['cleaned'] = train_X['all_text'].apply(lambda x: " ".join(re.sub("[^a-zA-Z']", " ", x).split()).lower())
#new_train['cleaned'] = new_train['cleaned'].apply(lambda s: " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(s)]))
#new_train['cleaned'] = new_train['cleaned'].apply(lambda x: " ".join([i for i in x.split() if i not in words]))
#new_train['cleaned'] = new_train['cleaned'].apply(lambda x: " ".join([stemmer.stem(i) for i in x.split()]))

In [101]:
valid_X['cleaned'] = valid_X['all_text'].apply(lambda x: " ".join(re.sub("[^a-zA-Z']", " ", x).split()).lower())
# valid_X['cleaned'] = valid_X['cleaned'].apply(lambda s: s.replace('!', ' ! '))
# valid_X['cleaned'] = valid_X['cleaned'].apply(lambda s: s.replace('?', ' ? '))
#valid_X['cleaned'] = valid_X['cleaned'].apply(lambda s: " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(s)]))
#valid_X['cleaned'] = valid_X['cleaned'].apply(lambda x: " ".join([i for i in x.split() if i not in words]))
#valid_X['cleaned'] = valid_X['cleaned'].apply(lambda x: " ".join([stemmer.stem(i) for i in x.split()]))

In [102]:
test['cleaned'] = test['all_text'].apply(lambda x: " ".join(re.sub("[^a-zA-Z']", " ", x).split()).lower())
# test['cleaned'] = test['cleaned'].apply(lambda s: s.replace('!', ' ! '))
# test['cleaned'] = test['cleaned'].apply(lambda s: s.replace('?', ' ? '))
#test['cleaned'] = test['cleaned'].apply(lambda s: " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(s)]))
#test['cleaned'] = test['cleaned'].apply(lambda x: " ".join([i for i in x.split() if i not in words]))
#test['cleaned'] = test['cleaned'].apply(lambda x: " ".join([stemmer.stem(i) for i in x.split()]))

In [122]:
vectorizer = TfidfVectorizer(stop_words=None, min_df=3, max_df=0.5)
pipeline = Pipeline([('vect', vectorizer),
#                     ('chi',  SelectKBest(chi2, k=all)),
                     ('clf', LinearRegression())])
pipeline.fit(train_X.loc[:10000, 'cleaned'], train_X.loc[:10000, 'price'])

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=3,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
..., ('clf', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))])

In [120]:
pred = pipeline.predict(valid_X.cleaned)
mean_squared_error(valid_X.price, pred)

4272254.6398827685

In [113]:
cvs_str = cross_val_score(pipeline, train_X.cleaned[:1000], train_X.price[:1000], cv=3, scoring='neg_mean_squared_error')

/usr/local/lib/python3.5/dist-packages/sklearn/feature_extraction/text.py:1547: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. float32 'dtype' will be converted to np.float64.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/feature_extraction/text.py:1547: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. float32 'dtype' will be converted to np.float64.
  UserWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/feature_extraction/text.py:1547: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. float32 'dtype' will be converted to np.float64.
  UserWarning)


In [114]:
cvs_str

array([-16650.45646743, -12689.16104487, -10815.21336694])

In [97]:
X_test_num[['id', 'price']].to_csv('answer_1.csv', index=False)